In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import docx2txt

docx_text = docx2txt.process('SEED-Fall-2024-Application-Template.docx')


# Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
text_chunks = text_splitter.split_text(docx_text)

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")

# Create and populate the vector store
vectorstore = Chroma.from_texts(
    embedding=embeddings, 
    persist_directory="./chroma_db"
)

vectorstore.delete_collection()
vectorstore.persist()

vectorstore = Chroma.from_texts(
    texts=text_chunks, 
    embedding=embeddings, 
    persist_directory="./chroma_db"
)

C:\Users\mukoo\AppData\Local\Temp\ipykernel_18684\2887218839.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")
c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Python312\Lib\site-packages\torch\cuda\__init__.py:758: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and

In [2]:
# Perform a sample similarity search
query = "Can I apply to the SEED program if I am part of a moving company?"
results = vectorstore.similarity_search(query, k=5)

print(f"Top 5 similar chunks for query '{query}':")
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result.page_content}\n")#[:200]}...")  # Print first 200 chars of each result

Top 5 similar chunks for query 'Can I apply to the SEED program if I am part of a moving company?':

1. THIS IS A TEMPLATE ONLY AND NOT THE NC IDEA APPLICATION.   
APPLICATIONS MUST BE COMPLETED AND SUBMITTED ONLINE VIA ncidea.my.site.com

NC IDEA SEED Grant Application 

Fall 2024

NOTES:

This document includes ONLY the free-form questions that are part of NC IDEA’s online application and can be used to draft your answers offline, as well as monitor character counts. This template may change slightly until the application opens.

Each application also requires information on company founders, funding details, conflicts of interest, revenue, and a few other items.  

Remember to give yourself adequate time to enter your answers into the online application found at the NC IDEA Community Portal at https://ncidea.my.site.com/, as you cannot upload this document template.

The character counts below include spaces.

You may only apply to either SEED or MICRO (not both) during a grant cycl

In [1]:
import os
import fitz  # PyMuPDF
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOllama
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

# Extract text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text")
    doc.close()
    return text

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")

# Initialize ChatOllama
llm = ChatOllama(model="llama3.1", device="cuda", temperature=0)

# Initialize the summarization chain
summarize_chain = load_summarize_chain(llm, chain_type="stuff")

# Initialize Chroma vector store
# vectorstore = Chroma(embedding_function=embeddings, persist_directory="./app_examples_db")
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chatbot_docs_db")

vectorstore.delete_collection()
# vectorstore = Chroma(embedding_function=embeddings, persist_directory="./app_examples_db")
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chatbot_docs_db")

# Process each PDF in the directory
# pdf_directory = "./pdf_documents"
pdf_directory = "./chatbot_pdf_documents"
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        pdf_text = extract_text_from_pdf(pdf_path)
        
        # Split the text into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=45000, chunk_overlap=50)
        text_chunks = text_splitter.split_text(pdf_text)
        
        # Generate summary
        documents = [Document(page_content=chunk) for chunk in text_chunks]
        summary = summarize_chain.run(documents)
        
        # Store summary in Chroma
        vectorstore.add_texts(texts=[summary], metadatas=[{"source": filename, "source_content": pdf_text}])

vectorstore.persist()


C:\Users\mukoo\AppData\Local\Temp\ipykernel_27012\1360290198.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")
c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\mukoo\AppData\Local\Temp\ipykernel_27012\1360290198.py:23: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1", device="cuda", temperature=0)
C:\Users\mukoo\AppData\Local\Temp\ipykernel_27012\1360290198.py:30: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chatbot_docs_db")
C:\Users\mukoo\App

In [3]:
import os
import fitz  # PyMuPDF
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOllama
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

# Extract text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text")
    doc.close()
    return text

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")

# Initialize Chroma vector store
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chatbot_docs_db")

vectorstore.delete_collection()
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chatbot_docs_db")

# Process each PDF in the directory
# pdf_directory = "./pdf_documents"
pdf_directory = "./chatbot_pdf_documents"
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        pdf_text = extract_text_from_pdf(pdf_path)
        
        # Split the text into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=45000, chunk_overlap=50)
        text_chunks = text_splitter.split_text(pdf_text)
        
        name = filename.split(".")[0]
        
        # Store summary in Chroma
        vectorstore.add_texts(texts=[name], metadatas=[{"source": filename, "source_content": pdf_text}])

vectorstore.persist()


In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")

# Load the existing Chroma database
vectorstore = Chroma(
    # persist_directory="./app_examples_db",
    persist_directory="./chatbot_docs_db",
    embedding_function=embeddings
)

# Perform a sample similarity search
query = "What problem am I trying to solve?"
doc_num = 1
results = vectorstore.similarity_search(query, k=doc_num)

print(f"Top 1 similar chunks for query '{query}':")
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result.page_content}\n")#[:200]}...")  # Print first 200 chars of each result
    print(f"\nSource: {result.metadata['source']}")  # Print source of the result
    print(f"\nSource Content: {result.metadata['source_content']}")  # Print source content of the result

Top 1 similar chunks for query 'What problem am I trying to solve?':

1. Business-Plan-Outline


Source: Business-Plan-Outline.pdf

Source Content: 1
sbtdc.org
Business Plan Outline
I. COVER PAGE 
II. TABLE OF CONTENTS 
III. EXECUTIVE SUMMARY 
The executive summary is a brief, one-page summary representing the various sections of your business plan. It is written last, but 
it is the most important part of your plan.  
IV. BACKGROUND INFORMATION
Note: If your business will be a seasonal business, make sure the seasonality is reflected in your narrative and financial projec-
tions with appropriate footnotes. 
A.	 Business concept 
a.	
Mission statement 
b.	
Goals and objectives of business 
c.	
Description of business 
B.	
Answer as many of the following questions as are appropriate: 
a.	
What business are you in? 
i.	 Merchandising 
ii.	 Manufacturing 
iii.	 Wholesale
iv.	 Service 
b.	
What is the nature of your product(s) or service(s) 
c.	
What will be special about your business? 
C